# Решение задачи:

Импортируем необходимые библиотеки:

In [ ]:
import numpy as np
from random import randint
import pandas as pd
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, LSTM, Input
from keras.optimizers import RMSprop, Adadelta, Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import utils
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from collections import defaultdict, Counter

Выполним скачивание архива с словарём для **англо-португальского** перевода. Архив я скачал с https://www.manythings.org/anki/ и поместил его в **Google Диск**:

In [ ]:
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=1p14kuZUHZw229qT0bSWO8qDKxo3xQ5t6" -O por-eng.zip

--2025-08-30 09:40:03--  https://docs.google.com/uc?export=download&id=1p14kuZUHZw229qT0bSWO8qDKxo3xQ5t6
Resolving docs.google.com (docs.google.com)... 142.250.4.139, 142.250.4.101, 142.250.4.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1p14kuZUHZw229qT0bSWO8qDKxo3xQ5t6&export=download [following]
--2025-08-30 09:40:04--  https://drive.usercontent.google.com/download?id=1p14kuZUHZw229qT0bSWO8qDKxo3xQ5t6&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.10.132, 2404:6800:4003:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.10.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6602160 (6.3M) [application/octet-stream]
Saving to: ‘por-eng.zip’

por-eng.zip         100%[===================>]   6.30M  17.5MB/s    in 0.4s 

Распакуем архив:

In [ ]:
!unzip -o por-eng.zip

Archive:  por-eng.zip
  inflating: _about.txt              
  inflating: por.txt                 


Определение констант в начале программы - это хорошая практика при разработке вашего программного обеспечения. Вы всегда быстро сможете изменить параметры обучения, а не вносить изменения по всему коду. Особенно, если обучение происходит на ограниченных ресурсах, то без подбора параметров вам не обойтись!  

In [ ]:
BATCH_SIZE = 256 # размер обучающего пакета
EPOCHS = 25 # число эпох обучения
LATENT_DIM = 256 # размерность латентного или контексного вектора
NUM_SAMPLES = 50000 # число примеров для обучения
FILE_NAME = "por.txt" # имя файла со словарем в архиве
SOS = '<start>' # токен начала последовательсти
EOS = '<end>' # токен окончания последовательсти

**Byte Pair Encoding (BPE)** — это алгоритм токенизации, который используется для разбиения текста на подслова (**subwords**), что особенно полезно для обработки редких слов и языков с богатой морфологией.

Метод `__init__` инициализирует объект BPE с параметрами для обучения. Метод `train` обучает **BPE**, создавая словарь токенов путем итеративного объединения наиболее частых пар. Метод `update_splits` обновляет разбиения слов (`self.splits`), заменяя указанную пару токенов (`lhs`, `rhs`) на их объединение. Метод `get_pairs_freq` вычисляет частотность всех пар соседних токенов в текущих разбиениях слов. Метод `tokenize` токенизирует входной текст, используя обученные объединения (`self.merges`).

In [ ]:
class BPE:
    def __init__(
        self,
        corpus: list[str],  # Корпус текстов для обучения токенизатора
        vocab_size: int,    # Желаемый размер словаря (количество токенов)
        max_iter: int | None = None,  # Максимальное количество итераций объединения
        debug: bool = False,  # Флаг для вывода отладочной информации
    ):
        # Сохраняем входные параметры
        self.corpus = corpus  # Список текстов (предложений) для обучения
        self.vocab_size = vocab_size  # Целевой размер словаря
        self.vocab = []  # Список токенов в словаре (начинается с символов, затем добавляются подслова)
        self.word_freq = Counter()  # Словарь для хранения частотности слов
        self.splits = {}  # Словарь вида {слово: [токены]}, например, {'highest': ['high', 'est</w>']}
        self.merges = {}  # Словарь объединений пар, например, {('high', 'est</w>'): 'highest'}
        self.max_iter = max_iter  # Ограничение на число итераций
        self.debug = debug  # Флаг для отладки
        self.special_tokens = ['<start>', '<end>']  # Специальные токены для начала и конца последовательности

    def train(self):
        # Подсчет частотности слов в корпусе
        for document in self.corpus:
            # Разбиваем каждое предложение на слова
            words = document.split()
            # Обновляем счетчик частотности слов
            self.word_freq += Counter(words)

        # Инициализация словаря и разбиений
        # Создаем начальный алфавит из всех уникальных символов в словах
        alphabet = set()
        for word in self.word_freq:
            alphabet |= set(list(word))  # Добавляем символы слова в множество
        alphabet.add("</w>")  # Добавляем токен конца слова
        alphabet |= set(self.special_tokens)  # Добавляем специальные токены
        self.vocab = list(alphabet)  # Преобразуем множество в отсортированный список
        self.vocab.sort()

        # Инициализируем разбиения слов (splits)
        for word in self.word_freq:
            if word in self.special_tokens:
                # Специальные токены остаются как есть
                self.splits[word] = [word]
            else:
                # Обычные слова разбиваем на символы и добавляем </w>
                self.splits[word] = list(word) + ["</w>"]

        # Выводим начальные разбиения для отладки, если debug=True
        if self.debug:
            print(f"Начальные разбиения: {self.splits}")

        # Счетчик итераций
        cnt = 0
        # Продолжаем, пока словарь не достигнет заданного размера
        while len(self.vocab) < self.vocab_size:
            # Проверяем ограничение на число итераций
            if self.max_iter and cnt >= self.max_iter:
                break

            # Находим частотность всех пар токенов
            pair_freq = self.get_pairs_freq()

            # Если пар больше нет, прерываем обучение
            if len(pair_freq) == 0:
                print("Нет доступных пар для объединения")
                break

            # Выбираем наиболее частую пару
            pair = max(pair_freq, key=pair_freq.get)

            # Обновляем разбиения слов, заменяя выбранную пару на объединенный токен
            self.update_splits(pair[0], pair[1])

            # Выводим обновленные разбиения для отладки
            if self.debug:
                print(f"Обновленные разбиения: {self.splits}")

            # Сохраняем объединение пары
            self.merges[pair] = pair[0] + pair[1]

            # Добавляем новый токен в словарь
            self.vocab.append(pair[0] + pair[1])

            # Выводим информацию о текущей паре и размере словаря для отладки
            if self.debug:
                print(
                    f"Наиболее частая пара ({max(pair_freq.values())} раз): "
                    f"{pair[0]}, {pair[1]}. Размер словаря: {len(self.vocab)}"
                )

            cnt += 1

    def update_splits(self, lhs: str, rhs: str):
        for word, word_split in self.splits.items():
            new_split = []
            cursor = 0
            # Проходим по токенам слова
            while cursor < len(word_split):
                # Если найдена пара (lhs, rhs), объединяем её
                if (
                    word_split[cursor] == lhs
                    and cursor + 1 < len(word_split)
                    and word_split[cursor + 1] == rhs
                ):
                    new_split.append(lhs + rhs)  # Добавляем объединенный токен
                    cursor += 2  # Пропускаем оба токена пары
                else:
                    new_split.append(word_split[cursor])  # Добавляем текущий токен
                    cursor += 1
            # Обновляем разбиение для слова
            self.splits[word] = new_split

    def get_pairs_freq(self) -> dict:
        pairs_freq = defaultdict(int)
        for word, freq in self.word_freq.items():
            split = self.splits[word]
            # Перебираем соседние токены в разбиении слова
            for i in range(len(split)):
                if i + 1 < len(split):
                    # Увеличиваем частоту пары на частоту слова
                    pairs_freq[(split[i], split[i + 1])] += freq

        return pairs_freq

    def tokenize(self, s: str) -> list[str]:
        splits = []
        # Разбиваем текст на слова
        for t in s.split():
            if t in self.special_tokens:
                # Специальные токены остаются без изменений
                splits.append([t])
            else:
                # Обычные слова разбиваем на символы и добавляем </w>
                splits.append(list(t) + ["</w>"])

        # Применяем все объединения из merges
        for lhs, rhs in self.merges:
            for idx, split in enumerate(splits):
                new_split = []
                cursor = 0
                while cursor < len(split):
                    # Если найдена пара (lhs, rhs), объединяем её
                    if (
                        cursor + 1 < len(split)
                        and split[cursor] == lhs
                        and split[cursor + 1] == rhs
                    ):
                        new_split.append(lhs + rhs)  # Добавляем объединенный токен
                        cursor += 2
                    else:
                        new_split.append(split[cursor])  # Добавляем текущий токен
                        cursor += 1
                # Проверяем, что объединение не изменило слово
                assert "".join(new_split) == "".join(split)
                splits[idx] = new_split

        # Объединяем все разбиения в один список токенов
        return sum(splits, [])

Функция `texts_to_sequences` преобразует список текстов (**texts**) в список последовательностей индексов, используя **BPE-токенизацию**. Для каждого текста она вызывает метод `bpe.tokenize` для разбиения на токены (**подслова**), а затем использует словарь `word_index` для преобразования каждого токена в соответствующий индекс, заменяя неизвестные токены на **0**.

*Результат* — список списков индексов, готовых для подачи в модель машинного обучения.

In [ ]:
def texts_to_sequences(texts, bpe, word_index):
    return [[word_index.get(token, 0) for token in bpe.tokenize(text)] for text in texts]

Прежде, чем "уходить в глубокую разработку", всегда полезно визуализировать данные, с которыми вам придется работать. В этом нам поможет библиотека **PANDAS**. Несмотря на то, что наш файл текстовый с разделителем "**табуляция**", мы всегда можем воспользоваться удобным методом `read_csv`. Данный метод умеет работать с любым текстовым файлом, главное правильно задать ему параметры для парсинга файла. В нашем случае, мы указываем разделитель табуляюцию: `sep='\t'`.

In [ ]:
df = pd.read_csv(FILE_NAME, sep='\t', header=None)
df.head()

,0,1,2
0,Go.,Vai.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Vá.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Hi.,Oi.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Run!,Corre!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


Часто полезно смотреть на данные не только с "головы" (первые записи в выборке), но и с "хвоста" (последние записи в выборке):

In [ ]:
df[NUM_SAMPLES-5:NUM_SAMPLES]

,0,1,2
49995,I'm extremely excited.,Eu estou extremamente empolgado.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
49996,I'm fairly optimistic.,Estou bastante otimista.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
49997,I'm fairly optimistic.,Eu estou bastante otimista.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
49998,I'm feeling confident.,Eu estou me sentindo confiante.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
49999,I'm feeling very good.,Eu estou me sentindo muito bem.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


Если бы мы смотрели на данные только с "головы", то решили бы, что работаем только со словами, а с "хвоста" мы видим сложные и длинные предложения.

Также мы видим, что данные состоят из трех столбцов, нас же интересуют только первых два, где первый столбик - входные данные, второй выходные для нашей будущей модели.

Далее сформируем обучаюющую выборку из `NUM_SAMPLES` примеров. По аналогии с чат-ботами мы сформируем массивы из входный фраз `questions` (вопросов) и выходных `answers` (ответов), а также определим токены начала последовательности `SOS` и токены окончания последовательности `EOS`:

In [ ]:
# Собираем вопросы и ответы в списки

questions = [] # список входных фраз
answers = []   # список ответных фраз

with open(FILE_NAME, "r", encoding="utf-8") as f: # открываем файл в режиме чтения
    lines = f.read().split("\n")                    # считываем весь файл, разделяя на строки

for line in lines[: min(NUM_SAMPLES, len(lines) - 1)]:
    # Разделяем строку по табам (входные данные, выходные и ненужный столбец)
    input_text, target_text, _ = line.split("\t")

    # В выходные данные для декодера добавляем токены SOS и EOS
    target_text = SOS + ' ' + target_text + ' ' + EOS
    questions.append(input_text)
    answers.append(target_text)

print("Число примеров:", len(answers))

#  Получим случайный вопрос и ответ
random_index = randint(0, len(questions)-1)
print(f'Вопрос : {questions[random_index]}') # Пример входной фразы
print(f'Ответ : {answers[random_index]}')    # Пример ответной фразы

Число примеров: 50000
Вопрос : I'll wait here.
Ответ : <start> Eu vou esperar aqui. <end>


Разбиваем данные на обучающую и тестовую выборки (80/20):

In [ ]:
questions_train, questions_test, answers_train, answers_test = train_test_split(questions, answers, test_size=0.2, random_state=42)

Этот код создает и обучает **BPE-токенизатор** для корпуса текстов. Сначала формируется корпус `corpus_for_bpe`, объединяя обучающие вопросы (`questions_train`) и ответы (`answers_train`), из которых удаляются токены <start> и <end> с пробелами. Затем создается объект **BPE** с заданным размером словаря 20,000 и обучается методом `bpe.train()`, который разбивает слова на подслова, итеративно объединяя наиболее частые пары токенов для построения словаря.

In [ ]:
# Создадим BPE токенизатор
corpus_for_bpe = questions_train + [a.replace(SOS + ' ', '').replace(' ' + EOS, '').strip() for a in answers_train]
bpe = BPE(corpus=corpus_for_bpe, vocab_size=20000, debug=False)
bpe.train()

Этот код создает словари для преобразования токенов в индексы и обратно после обучения **BPE-токенизатора**. `word_index` сопоставляет каждому токену из `bpe.vocab` уникальный индекс (начиная с 1), а `index_to_word` — обратное сопоставление индексов токенам. Далее, `vocabularyItems` сохраняет пары (токен, индекс) как список, а `vocabularySize` вычисляется как размер `word_index` плюс 1 (для нулевого индекса, обозначающего неизвестные токены):

In [ ]:
# Создаем словари после обучения BPE
word_index = {token: i+1 for i, token in enumerate(bpe.vocab)}
index_to_word = {i: token for token, i in word_index.items()}

# Список с содержимым словаря
vocabularyItems = list(word_index.items())

# Размер словаря
vocabularySize = len(word_index) + 1

Следующий код преобразует обучающие и тестовые вопросы (`questions_train`, `questions_test`) и ответы (`answers_train`, `answers_test`) в последовательности индексов токенов. Используя функцию `texts_to_sequences`, он токенизирует каждый текст с помощью **BPE-токенизатора** (**bpe**) и преобразует токены в индексы из словаря `word_index`:

In [ ]:
# Векторизируем входные и выходные фразы (вопросы и ответы) для train
tokenizedQuestions_train = texts_to_sequences(questions_train, bpe, word_index)
tokenizedAnswers_train = texts_to_sequences(answers_train, bpe, word_index)

# Векторизируем для test
tokenizedQuestions_test = texts_to_sequences(questions_test, bpe, word_index)
tokenizedAnswers_test = texts_to_sequences(answers_test, bpe, word_index)

Далее подготавливаются токенизированные данные для модели машинного перевода, вычисляя максимальную длину последовательностей, выравнивая их длину нулями, преобразуя в массивы **NumPy** и выводя примеры для отладки. Сначала определяется максимальная длина последовательностей вопросов (`maxLenQuestions`) и ответов (`maxLenAnswers`) на основе обучающих и тестовых данных для согласованности. Затем функция `pad_sequences` выравнивает все последовательности до этих длин, добавляя нули в конец, а данные преобразуются в массивы **NumPy** (`encoderForInput`, `decoderForInput`) для подачи в модель, с выводом примеров вопроса, ответа и их векторизаций для проверки:

In [ ]:
# Получаем длину самой длинной фразы (на основе всех данных для consistency)
maxLenQuestions = max([len(x) for x in tokenizedQuestions_train + tokenizedQuestions_test])
maxLenAnswers = max([len(x) for x in tokenizedAnswers_train + tokenizedAnswers_test])

# Делаем последовательности одной длины, заполняя нулями более короткие фразы (отдельно для вопросов и ответов)
paddedQuestions_train = pad_sequences(tokenizedQuestions_train, maxlen=maxLenQuestions, padding='post')
paddedAnswers_train = pad_sequences(tokenizedAnswers_train, maxlen=maxLenAnswers, padding='post')

paddedQuestions_test = pad_sequences(tokenizedQuestions_test, maxlen=maxLenQuestions, padding='post')
paddedAnswers_test = pad_sequences(tokenizedAnswers_test, maxlen=maxLenAnswers, padding='post')

# Создаем numpy массив для входа в кодировщик (train)
encoderForInput = np.array(paddedQuestions_train)

# Создаем numpy массив для входа в декодировщик (train)
decoderForInput = np.array(paddedAnswers_train)

#  Получим случайный вопрос и ответ
random_index = randint(0, len(questions_train)-1)

# Выведем фрагмент и размер словаря
print( f'Фрагмент словаря : {bpe.vocab[:50]}')
print( f'Размер словаря   : {vocabularySize}')

# Примеры данных для вопросов
print(f'Пример вопроса                         : {questions_train[random_index]}')
print(f'Пример векторизации вопроса            : {encoderForInput[random_index]}')
print(f'Размер векторизованного вопроса        : {encoderForInput.shape}')
print(f'Новая длина вопроса                    : {maxLenQuestions}')

# Примеры данных для ответов
print(f'Пример ответа                         : {answers_train[random_index]}')
print(f'Пример векторизации ответа            : {decoderForInput[random_index]}')
print(f'Размер векторизованного ответа        : {decoderForInput.shape}')
print(f'Новая длина ответа                    : {maxLenAnswers}')

Фрагмент словаря : ['!', '"', '#', '$', '%', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '</w>', '<end>', '<start>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V']
Размер словаря   : 20001
Пример вопроса                         : I'd like to dance.
Пример векторизации вопроса            : [1170  303  177 3830    0    0    0    0    0]
Размер векторизованного вопроса        : (40000, 9)
Новая длина вопроса                    : 9
Пример ответа                         : <start> Eu gostaria de dançar. <end>
Пример векторизации ответа            : [  27  129 2353  148 3676   26    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
Размер векторизованного ответа        : (40000, 20)
Новая длина ответа                    : 20


Затем разбиваем текст ответов на последовательности индексов для **train** и избавляемся от тега **SOS**:

In [ ]:
tokenizedAnswers_train = texts_to_sequences(answers_train, bpe, word_index)

for i in range(len(tokenizedAnswers_train)) :
    tokenizedAnswers_train[i] = tokenizedAnswers_train[i][1:]

Делаем ответы одной длины и сохраняем в виде массива **Numpy**:

In [ ]:
paddedAnswers_train = pad_sequences(tokenizedAnswers_train, maxlen=maxLenAnswers , padding='post')

decoderForOutput = np.array(paddedAnswers_train)

# Примеры данных для ответов на выходе декодировщика
print(f'Пример выходных данных декодировщика                : {answers_train[random_index]}')
print(f'Пример векторизации выходных данных декодировщика   : {decoderForOutput[random_index]}')
print(f'Размер выходных данных декодировщика                : {decoderForOutput.shape}')
print(f'Новая длина выходных данных декодировщика           : {maxLenAnswers}')

Пример выходных данных декодировщика                : <start> Eu gostaria de dançar. <end>
Пример векторизации выходных данных декодировщика   : [ 129 2353  148 3676   26    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
Размер выходных данных декодировщика                : (40000, 20)
Новая длина выходных данных декодировщика           : 20


Опишем архитектуру кодировщика:

In [ ]:
encoderInputs = Input(shape=(None , ))                                                    # добавим входной слой
encoderEmbedding = Embedding(vocabularySize, LATENT_DIM , mask_zero=True)(encoderInputs)  # добавим эмбеддинг
encoderOutputs, state_h , state_c = LSTM(LATENT_DIM, return_state=True)(encoderEmbedding) # добавим LSTM
encoderStates = [state_h, state_c]

Опишем архитектуру декодировщика:

In [ ]:
decoderInputs = Input(shape=(None, ))                                                       # добавим входной слой
decoderEmbedding = Embedding(vocabularySize, LATENT_DIM, mask_zero=True) (decoderInputs)    # добавим эмбеддинг
decoderLSTM = LSTM(LATENT_DIM, return_state=True, return_sequences=True)                    # добавим LSTM слой
decoderOutputs , _ , _ = decoderLSTM(decoderEmbedding, initial_state=encoderStates)         # погоним выход embedding через LSTM (вектора состояний нас уже не интересуют)
decoderDense = Dense(vocabularySize, activation='softmax')                                  # создадим dense слой с функцией активации softmax и длиной словаря, созданного токенизатором
output = decoderDense (decoderOutputs)

Выполним сборку модели, её компиляцию и обучение:

In [ ]:
model = Model([encoderInputs, decoderInputs], output)
model.compile(optimizer=RMSprop(), loss='sparse_categorical_crossentropy')
model.fit([encoderForInput , decoderForInput], decoderForOutput, batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 31s 162ms/step - loss: 7.2703
Epoch 2/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 165ms/step - loss: 4.6943
Epoch 3/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 169ms/step - loss: 4.5096
Epoch 4/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 172ms/step - loss: 4.3474
Epoch 5/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 170ms/step - loss: 4.1848
Epoch 6/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 171ms/step - loss: 4.0222
Epoch 7/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 171ms/step - loss: 3.8672
Epoch 8/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 171ms/step - loss: 3.7559
Epoch 9/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 171ms/step - loss: 3.6353
Epoch 10/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 171ms/step - loss: 3.5435
Epoch 11/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 27s 171ms/step - loss: 3.4447
Epoch 12/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 171ms/step - loss: 3.3505
Epoch 13/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 27s 173ms/step - loss: 3.2512
Epoch 14/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 27s 170ms/step - loss: 3.1672
Epoch 15/25
157

Этот код создает две отдельные модели для этапа инференса в машинном переводе: модель кодировщика и модель декодировщика. Модель кодировщика (`encoderModel`) принимает закодированные вопросы (`encoderInputs`) и возвращает их контекстные состояния (`state_h`, `state_c`) для передачи в декодировщик. Модель декодировщика (`decoderModel`) принимает входные токены (`decoderInputs`), эмбеддинги (`decoderEmbedding`) и начальные состояния (`decoderStatesInputs`), пропускает их через **LSTM-слой**, обновляет состояния (`state_h`, `state_c`) и предсказывает следующий токен через полносвязный слой с **softmax** (`decoderDense`), возвращая предсказания и новые состояния:

In [ ]:
# Создадим модель кодировщика
# На входе будут закодированные вопросы, на выходе состояния state_h, state_c
encoderModel = Model(encoderInputs, encoderStates)

# Создадим модель декодировщика
decoderStateInput_h = Input(shape=(LATENT_DIM,)) # входной слой для state_h
decoderStateInput_c = Input(shape=(LATENT_DIM,)) # входной слой для state_c

# Соберем оба входа вместе
decoderStatesInputs = [decoderStateInput_h, decoderStateInput_c]

# Берём ответы, прошедшие через эмбединг, вместе с состояниями и подаём LSTM cлою
decoderOutputs, state_h, state_c = decoderLSTM(decoderEmbedding, initial_state=decoderStatesInputs)

# LSTM даст нам новые состояния
decoderStates = [state_h, state_c]

# И ответы, которые мы пропустим через полносвязный слой с софтмаксом
decoderOutputs = decoderDense(decoderOutputs)

# Определим модель декодировщика
decoderModel = Model([decoderInputs] + decoderStatesInputs, [decoderOutputs] + decoderStates)

Этот код определяет функцию `predict_translation`, которая выполняет перевод входной фразы (`my_question`) с английского на португальский, используя обученную модель энкодера-декодера. Функция токенизирует входное предложение с помощью **BPE**, преобразует его в последовательность индексов, получает контекстные состояния от энкодера, а затем итеративно генерирует перевод, предсказывая следующий токен через декодер, пока не встретится токен <end> или не превысится максимальная длина ответа, после чего токены объединяются в строку с заменой </w> на пробелы.

*Результат* — строка переведенного текста.

In [ ]:
# Функция для предсказания перевода
def predict_translation(my_question):
    # Токенизируем предложение
    tokens_list = texts_to_sequences([my_question], bpe, word_index)[0]

    # Зафиксируем длину последовательности, дополнив нулями
    question_token = pad_sequences([tokens_list], maxlen=maxLenQuestions , padding='post')

    targetSeq = np.zeros((1, 1))                        # объявляем последовательность
    targetSeq[0, 0] = word_index['<start>']     # на начальном этапе последовательность содержит только токен начала последовательности
    stop = False                                        # признак окончания генерации последоватнльности токенов
    decoded_tokens = []                                 # список с результатами предсказания
    statesValues = encoderModel.predict(question_token, verbose=0) # получение контектного вектора из кодировщика

    # пока не сработало стоп-условие
    while not stop:
        # В модель декодера подадим пустую последовательность со словом 'start' и состояния
        decOutputs , h , c = decoderModel.predict([targetSeq] + statesValues, verbose=0)
        # Получим индекс предсказанного слова.
        predictIndex = np.argmax( decOutputs[0, 0, :])

        # Создаем переменную для хранения предсказанного слова
        predictWord = index_to_word.get(predictIndex, None)
        if predictWord is None:
            stop = True
            continue

        # Добавляем к списку
        decoded_tokens.append(predictWord)

        # Если найденное слово является признаком окончания генерации '<end>' или ответ превышает максимальную длину ответа, то останавливаем генерацию
        if predictWord == '<end>' or len(decoded_tokens) > maxLenAnswers:
            stop = True # устанавливаем признак окончания генерации

        # Обновляем входной токен для следующего шага генерации
        targetSeq = np.zeros((1, 1))
        targetSeq[0, 0] = predictIndex

        # Обновляем состояния ячейки и переходим к следующему шагу цикла
        statesValues = [h, c]

    # Детокенизируем
    decoded_answer = ''.join(decoded_tokens).replace('</w>', ' ').replace('<end>', '').strip()

    return decoded_answer

Финальный код демонстрирует работу модели перевода на **10** случайных примерах из тестовой выборки. Он выбирает **10** случайных индексов из `questions_test`, для каждого индекса получает английскую фразу, истинный португальский перевод (очищенный от токенов <start> и <end>), и предсказанный перевод с помощью функции `predict_translation`, затем сохраняет их в списки и создает таблицу `results_df` с колонками "**Английская фраза**", "**Португальский перевод**" и "**Предсказанный перевод**" для вывода результатов:

In [ ]:
# Демонстрация на 10 примерах из тестовой выборки
num_examples = 10
test_indices = np.random.choice(len(questions_test), num_examples, replace=False)

eng_phrases = []
true_por = []
pred_por = []

for idx in test_indices:
    eng = questions_test[idx]
    true = answers_test[idx].replace(SOS + ' ', '').replace(' ' + EOS, '').strip()
    pred = predict_translation(eng)

    eng_phrases.append(eng)
    true_por.append(true)
    pred_por.append(pred)

# Сформируем таблицу
results_df = pd.DataFrame({
    'Английская фраза': eng_phrases,
    'Португальский перевод': true_por,
    'Предсказанный перевод': pred_por
})

print(results_df)

        Английская фраза     Португальский перевод    Предсказанный перевод
0      Tom was very sad.  Tom estava muito triste.  Tom estava muito feliz.
1    I want this guitar.     Eu quero este violão.  Eu quero mais de volta.
2   It finally happened.     Finalmente aconteceu.           Isso foi isso.
3            He gave up.             Ele desistiu.         Ele me foi tudo.
4   I hurt myself today.        Me machuquei hoje.         Eu me sinto bem.
5  Tom was never caught.       Tom nunca foi pego.     Tom estava com fome.
6   This site is useful.         Este site é útil.            Esta é a meu.
7        I'm quite sure.      Tenho quase certeza.   Estou cansado de novo.
8     Let's have dinner.             Vamos jantar.        Vamos nos ajudar.
9       I made that one.            Eu fiz aquele.    Eu me vou fazer isso.
